In [ ]:
!wget "https://pancanatlas.xenahubs.net/download/TCGA_phenotype_denseDataOnlyDownload.tsv.gz"

--2022-10-17 16:56:47--  https://pancanatlas.xenahubs.net/download/TCGA_phenotype_denseDataOnlyDownload.tsv.gz
Resolving pancanatlas.xenahubs.net (pancanatlas.xenahubs.net)... 52.45.45.5, 18.232.241.34, 52.5.197.193, ...
Connecting to pancanatlas.xenahubs.net (pancanatlas.xenahubs.net)|52.45.45.5|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://tcga-pancan-atlas-hub.s3.us-east-1.amazonaws.com:443/download/TCGA_phenotype_denseDataOnlyDownload.tsv.gz [following]
--2022-10-17 16:56:47--  https://tcga-pancan-atlas-hub.s3.us-east-1.amazonaws.com/download/TCGA_phenotype_denseDataOnlyDownload.tsv.gz
Resolving tcga-pancan-atlas-hub.s3.us-east-1.amazonaws.com (tcga-pancan-atlas-hub.s3.us-east-1.amazonaws.com)... 3.5.11.161
Connecting to tcga-pancan-atlas-hub.s3.us-east-1.amazonaws.com (tcga-pancan-atlas-hub.s3.us-east-1.amazonaws.com)|3.5.11.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61165 (60K) [text/tab-separ

In [ ]:
!gunzip TCGA_phenotype_denseDataOnlyDownload.tsv.gz

In [ ]:
!wget "https://legacy.xenahubs.net/download/TCGA.PANCAN.sampleMap/HiSeqV2.gz"

--2022-10-17 16:56:47--  https://legacy.xenahubs.net/download/TCGA.PANCAN.sampleMap/HiSeqV2.gz
Resolving legacy.xenahubs.net (legacy.xenahubs.net)... 54.83.19.224
Connecting to legacy.xenahubs.net (legacy.xenahubs.net)|54.83.19.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 513041354 (489M) [application/gzip]
Saving to: ‘HiSeqV2.gz’

HiSeqV2.gz          100%[===================>] 489.27M  16.3MB/s    in 30s     

2022-10-17 16:57:17 (16.4 MB/s) - ‘HiSeqV2.gz’ saved [513041354/513041354]



In [ ]:
!gunzip 'HiSeqV2.gz'

In [ ]:
import pandas as pd
import h5py
import numpy as np
import progressbar

In [ ]:
data = 'HiSeqV2'
labels = 'TCGA_phenotype_denseDataOnlyDownload.tsv'
dbPath = 'data.h5'
verbose = False

In [ ]:

print('Loading data ... Patience.')
df = pd.read_csv(data, sep='\t').transpose()

print('Loading labels ...')
labeldf = pd.read_csv(labels, sep = '\t')

print('Housekeeping ...')
df.columns = df.iloc[0]
df = df.drop('Sample', axis = 0)

labeldf = labeldf.set_index('sample')

# dimensions: 10459 x 20530

nTotal = df.shape[0]    #10459
nFeat = df.shape[1]     #20530

print('Total Number of samples: '+ str(nTotal))
print('Features (RNASeq) per sample: ' + str(nFeat))

print('Diseases to predict: ')

diseases = labeldf._primary_disease.unique()

for disease in diseases:
    print(disease)

# Defining Categorical values for each disease

diseasedict = {
    'skin cutaneous melanoma':0, 'thyroid carcinoma':1, 'sarcoma':2,
    'prostate adenocarcinoma':3, 'pheochromocytoma & paraganglioma':4,
    'pancreatic adenocarcinoma':5, 'head & neck squamous cell carcinoma':6,
    'esophageal carcinoma':7, 'colon adenocarcinoma':8,
    'cervical & endocervical cancer':9, 'breast invasive carcinoma':10,
    'bladder urothelial carcinoma':11, 'testicular germ cell tumor':12,
    'kidney papillary cell carcinoma':13, 'kidney clear cell carcinoma':14,
    'acute myeloid leukemia':15, 'rectum adenocarcinoma':16,
    'ovarian serous cystadenocarcinoma':17, 'lung adenocarcinoma':18,
    'liver hepatocellular carcinoma':19,
    'uterine corpus endometrioid carcinoma':20, 'glioblastoma multiforme':21,
    'brain lower grade glioma':22, 'uterine carcinosarcoma':23, 'thymoma':24,
    'stomach adenocarcinoma':25, 'diffuse large B-cell lymphoma':26,
    'lung squamous cell carcinoma':27, 'mesothelioma':28,
    'kidney chromophobe':29, 'uveal melanoma':30, 'cholangiocarcinoma':31,
    'adrenocortical cancer':32
}

print('Creating Database File at : ' + dbPath)
db = h5py.File(dbPath, mode = 'w')

print('Setting up Database')
db.create_dataset("name", (nTotal,), np.dtype('|S16'))
db.create_dataset("RNASeq", (nTotal, nFeat), np.float32)
db.create_dataset("label", (nTotal,), np.uint8)

idx = 0

print('Writing ' + str(nTotal) + ' samples to Dataset')

for index,row in progressbar.progressbar(df.iterrows(), redirect_stdout=True):
    try:
        data = labeldf.loc[index]
        if(verbose):
            print('Processing '+ str(idx) + ' of ' + str(nTotal) + ' : ' + index + '\t disease: \t' + str(data[2]))
        db["name"][idx] = np.asarray(index, dtype = np.dtype('|S16'))
        db["RNASeq"][idx] = np.asarray(row, dtype = np.float32)
        db["label"][idx] = np.uint8(diseasedict[data[2]])
        idx = idx + 1
    except:
        print("Error: Cannot find label")
        continue

print('Closing Database ..')
db.close()
print('Complete!')

Loading data ... Patience.
Loading labels ...
Housekeeping ...


\ | #                                                | 25 Elapsed Time: 0:00:00

Total Number of samples: 10459
Features (RNASeq) per sample: 20530
Diseases to predict: 
skin cutaneous melanoma
thyroid carcinoma
sarcoma
prostate adenocarcinoma
pheochromocytoma & paraganglioma
pancreatic adenocarcinoma
head & neck squamous cell carcinoma
esophageal carcinoma
colon adenocarcinoma
cervical & endocervical cancer
breast invasive carcinoma
bladder urothelial carcinoma
testicular germ cell tumor
kidney papillary cell carcinoma
kidney clear cell carcinoma
acute myeloid leukemia
rectum adenocarcinoma
ovarian serous cystadenocarcinoma
lung adenocarcinoma
liver hepatocellular carcinoma
uterine corpus endometrioid carcinoma
glioblastoma multiforme
brain lower grade glioma
uterine carcinosarcoma
thymoma
stomach adenocarcinoma
diffuse large B-cell lymphoma
lung squamous cell carcinoma
mesothelioma
kidney chromophobe
uveal melanoma
cholangiocarcinoma
adrenocortical cancer
Creating Database File at : data.h5
Setting up Database
Writing 10459 samples to Dataset


| |                                       #       | 10458 Elapsed Time: 0:00:41


Closing Database ..
Complete!


In [ ]:
del df

In [ ]:
db = h5py.File(dbPath, mode = 'r')
X = db["RNASeq"][...]
y = db["label"][...]

In [ ]:
X.shape

(10459, 20530)

In [ ]:
y.max()

32

In [ ]:
print(X.shape)
print(y.shape)

(10459, 20530)
(10459,)


In [ ]:
from sklearn.manifold import TSNE as TSNE
tsne = TSNE(n_jobs=4, n_components=2, verbose = 1)
Y  = tsne.fit_transform(X)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 10459 samples in 0.191s...
[t-SNE] Computed neighbors for 10459 samples in 88.612s...
[t-SNE] Computed conditional probabilities for sample 1000 / 10459
[t-SNE] Computed conditional probabilities for sample 2000 / 10459
[t-SNE] Computed conditional probabilities for sample 3000 / 10459
[t-SNE] Computed conditional probabilities for sample 4000 / 10459
[t-SNE] Computed conditional probabilities for sample 5000 / 10459
[t-SNE] Computed conditional probabilities for sample 6000 / 10459
[t-SNE] Computed conditional probabilities for sample 7000 / 10459
[t-SNE] Computed conditional probabilities for sample 8000 / 10459
[t-SNE] Computed conditional probabilities for sample 9000 / 10459
[t-SNE] Computed conditional probabilities for sample 10000 / 10459
[t-SNE] Computed conditional probabilities for sample 10459 / 10459
[t-SNE] Mean sigma: 47.301785
[t-SNE] KL divergence after 250 iterations with early exaggeration: 69.763092
[t-SNE] K

In [ ]:
diseasedict = {
    'skin cutaneous melanoma':0, 'thyroid carcinoma':1, 'sarcoma':2,
    'prostate adenocarcinoma':3, 'pheochromocytoma & paraganglioma':4,
    'pancreatic adenocarcinoma':5, 'head & neck squamous cell carcinoma':6,
    'esophageal carcinoma':7, 'colon adenocarcinoma':8,
    'cervical & endocervical cancer':9, 'breast invasive carcinoma':10,
    'bladder urothelial carcinoma':11, 'testicular germ cell tumor':12,
    'kidney papillary cell carcinoma':13, 'kidney clear cell carcinoma':14,
    'acute myeloid leukemia':15, 'rectum adenocarcinoma':16,
    'ovarian serous cystadenocarcinoma':17, 'lung adenocarcinoma':18,
    'liver hepatocellular carcinoma':19,
    'uterine corpus endometrioid carcinoma':20, 'glioblastoma multiforme':21,
    'brain lower grade glioma':22, 'uterine carcinosarcoma':23, 'thymoma':24,
    'stomach adenocarcinoma':25, 'diffuse large B-cell lymphoma':26,
    'lung squamous cell carcinoma':27, 'mesothelioma':28,
    'kidney chromophobe':29, 'uveal melanoma':30, 'cholangiocarcinoma':31,
    'adrenocortical cancer':32
}


In [ ]:
keyslist = list(diseasedict.keys())
valueslist = list(diseasedict.values())

cancers = []

for classno in y:
  cancers.append(keyslist[valueslist.index(classno)]) 

In [ ]:
tsne = pd.DataFrame(Y, columns = ["tsne1", "tsne2"])
cancers = pd.DataFrame(cancers, columns = ["cancer"])
tsne = pd.concat([tsne,cancers], axis = 1, sort = False)
tsne = tsne.sort_values(by = "cancer")

In [ ]:
!pip install plotly_express

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import plotly_express as px

figx = px.scatter(
    tsne,
    x="tsne1",
    y="tsne2",
    color="cancer",
    hover_name="cancer",
    width=970,
    height=500,
    template="ggplot2",
    color_discrete_sequence= px.colors.qualitative.Alphabet,
    #facet_col="group_label",
    size_max=0.1,
)

figx.show()

In [ ]:
!pip install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.2 MB/s 
     |████████████████████████████████| 1.1 MB 29.4 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=51e103d1e34395fb2d01938520571ae98c82240afa2697c123b4803c660454d8
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=65204504a1c1e3a902efaf3b44f4f35826f88759f091aa6f7e3826b91137a2be
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dcec71bd5c669f6f574db3113513696b98f3f9b51f496c
Successfully built umap-learn pynndescent


In [ ]:
from umap import UMAP

In [ ]:
umap = UMAP()

Y  = umap.fit_transform(X)

In [ ]:
umap_pl = pd.DataFrame(Y, columns = ["umap1", "umap2"])
umap_pl = pd.concat([umap_pl,cancers], axis = 1, sort = False)
umap_pl = umap_pl.sort_values(by = "cancer")

In [ ]:
figx = px.scatter(
    umap_pl,
    x="umap1",
    y="umap2",
    color="cancer",
    hover_name="cancer",
    width=970,
    height=500,
    template="ggplot2",
    color_discrete_sequence= px.colors.qualitative.Alphabet,
    #facet_col="group_label",
    size_max=0.1,
)

figx.show()

In [ ]:
X

array([[10.9576,  4.8099,  0.4657, ...,  9.6465,  0.    ,  9.4848],
       [11.0186,  5.3847,  0.    , ..., 13.0045,  7.0466, 10.3411],
       [ 9.7106,  2.8888,  0.4192, ...,  9.2958,  0.4192,  9.745 ],
       ...,
       [ 9.6575,  8.9521,  0.4791, ..., 10.9969,  9.3046, 10.2187],
       [11.7589,  3.7591,  0.    , ..., 13.3772,  6.4848,  9.8594],
       [11.525 ,  3.9462,  0.    , ..., 13.0054,  5.9179, 10.2727]],
      dtype=float32)

In [ ]:
y

array([22, 19,  9, ..., 18, 19, 19], dtype=uint8)

In [ ]:
label = [1 if x == 3 else 0 for x in y]

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
cls = LogisticRegression()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.33, random_state=42)

In [ ]:
cls.fit(X_train,y_train)

LogisticRegression()

In [ ]:
predictions = cls.predict(X_test)

In [ ]:
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print(accuracy_score(y_test,predictions))

1.0
